<a href="https://colab.research.google.com/github/Pakhi27/Pose-Estimation-Computer-Vision/blob/main/Pose_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pose Dataset

PoseNet Model

In [30]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np

# Load the PoseNet model (MoveNet SinglePose Lightning)
model = hub.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')

# Define keypoint pairs for drawing the skeleton
# These are based on the MoveNet output keypoints
KEYPOINT_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),   # Nose to shoulders
    (0, 5), (5, 6), (6, 7),           # Left shoulder to left elbow and wrist
    (0, 11), (11, 12), (12, 13),      # Right shoulder to right elbow and wrist
    (0, 8), (8, 9), (9, 10),          # Right hip to knee and ankle
    (0, 14), (14, 15), (15, 16)       # Left hip to knee and ankle
]

def detect_pose(image):
    # Preprocess the image
    input_image = tf.image.resize_with_pad(np.expand_dims(image, axis=0), 192, 192)
    input_image = tf.cast(input_image, dtype=tf.int32)

    # Run model inference
    outputs = model.signatures['serving_default'](input_image)

    # Output keypoints and confidence scores
    keypoints = outputs['output_0'].numpy()
    return keypoints

def draw_skeleton(image, keypoints, confidence_threshold=0.4):
    # The keypoints are in the format [y, x, confidence]
    keypoints = keypoints[0][0]
    h, w, _ = image.shape

    # Draw lines between keypoints
    for connection in KEYPOINT_CONNECTIONS:
        start_idx, end_idx = connection
        start_keypoint = keypoints[start_idx]
        end_keypoint = keypoints[end_idx]

        # Unpack the coordinates and confidence
        start_y, start_x, start_conf = start_keypoint
        end_y, end_x, end_conf = end_keypoint

        # Draw the line only if both keypoints have high confidence
        if start_conf > confidence_threshold and end_conf > confidence_threshold:
            # Convert normalized coordinates to pixel values
            start_px, start_py = int(start_x * w), int(start_y * h)
            end_px, end_py = int(end_x * w), int(end_y * h)
            # Draw the line between the points
            cv2.line(image, (start_px, start_py), (end_px, end_py), (0, 255, 0), 2)

    # Draw keypoints on the image
    for i, keypoint in enumerate(keypoints):
        y, x, confidence = keypoint
        if confidence > confidence_threshold:
            px, py = int(x * w), int(y * h)
            cv2.circle(image, (px, py), 5, (0, 0, 255), -1)

    return image

# Load your image
image = cv2.imread('/content/pose_2.jpeg')

# Detect the pose keypoints
keypoints = detect_pose(image)

# Draw the skeleton on the image
image_with_skeleton = draw_skeleton(image.copy(), keypoints)

# Save the image with skeleton
cv2.imwrite('/content/output_image_with_skeleton.jpg', image_with_skeleton)

print("Image saved with skeleton: /content/output_image_with_skeleton.jpg")


Image saved with skeleton: /content/output_image_with_skeleton.jpg


MoveNet Model-Tensorflow

In [36]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np

# Load the MoveNet model
model = hub.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')

def detect_pose(image):
    # Preprocess the image
    input_image = tf.image.resize_with_pad(np.expand_dims(image, axis=0), 192, 192)
    input_image = tf.cast(input_image, dtype=tf.int32)

    # Run model inference
    outputs = model.signatures['serving_default'](input_image)

    # Output keypoints and confidence scores
    keypoints = outputs['output_0'].numpy()
    return keypoints

def draw_keypoints(image, keypoints, confidence_threshold=0.3):
    height, width, _ = image.shape
    for keypoint in keypoints[0, 0, :, :]:
        y, x, confidence = keypoint
        if confidence > confidence_threshold:
            cv2.circle(image, (int(x * width), int(y * height)), 5, (0, 255, 0), -1)
    return image

# Load your image
image = cv2.imread('/content/pose_2.jpeg')

# Detect the pose
keypoints = detect_pose(image)

# Draw keypoints
image_with_keypoints = draw_keypoints(image.copy(), keypoints)

# Save the image with pose keypoints
cv2.imwrite('/content/output_pose_tensorflow.png', image_with_keypoints)

print("Image saved as '/content/output_pose_tensorflow.png'")


Image saved as '/content/output_pose_tensorflow.png'


OpenPose (Alternative Method):BlazePose Setup

In [34]:
!pip install mediapipe


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [37]:
import cv2
import mediapipe as mp

# Initialize BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Load image
image = cv2.imread('/content/pose_2.jpeg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process the image to detect pose
results = pose.process(image_rgb)

# Draw landmarks on the image
if results.pose_landmarks:
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

# Save the result image
cv2.imwrite("/content/output_pose_mediapipe.png", image)

print("Image saved as '/content/output_pose_mediapipe.png'")


Image saved as '/content/output_pose_mediapipe.png'
